<a href="https://colab.research.google.com/github/nonyeezeh/Research-Project-Code/blob/main/NN_Dense_1_20_Relu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

In [1]:
pip install pgmpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 17.7 MB/s eta 0:00:00


In [2]:
import numpy as np
import pandas as pd
from pgmpy.models import BayesianNetwork
from pgmpy.models import BayesianModel
from pgmpy.factors.discrete import TabularCPD
from pgmpy.sampling import BayesianModelSampling
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from tensorflow.keras import layers, models
from pgmpy.estimators import HillClimbSearch, BicScore, AICScore, MaximumLikelihoodEstimator
from pgmpy.inference import VariableElimination
from sklearn.metrics import accuracy_score
from scipy.stats import entropy
import os
import matplotlib.pyplot as plt

# Bayesian Network Data Generation 1000, 2000, ..., 10000 Samples (dense)

In [3]:
# Define the mappings for IR, EI, SP
ir_map = {0: 'low', 1: 'medium', 2: 'high'}
ei_map = {0: 'poor', 1: 'average', 2: 'good'}
sp_map = {0: 'decrease', 1: 'stable', 2: 'increase'}

# Define the dense Bayesian Network
dense_model = BayesianNetwork([('IR', 'EI'), ('EI', 'SP'), ('IR', 'SP')])

# Function to generate CPDs
def generate_cpds():
    ir_probs = np.random.rand(3)
    ir_probs /= ir_probs.sum()  # Normalize to make it a valid probability distribution

    ei_given_ir_probs = np.random.rand(3, 3)
    ei_given_ir_probs /= ei_given_ir_probs.sum(axis=0, keepdims=True)

    sp_probs = np.random.rand(3, 3, 3)
    sp_probs /= sp_probs.sum(axis=0, keepdims=True)

    sp_probs_reshaped = sp_probs.reshape(3, -1)

    return ir_probs, ei_given_ir_probs, sp_probs_reshaped

# Save probabilities in a single CSV file
def save_probabilities(ir_probs, ei_probs, sp_probs, filename):
    # Create a DataFrame for IR probabilities
    ir_df = pd.DataFrame({
        'IR_State': ['low', 'medium', 'high'],
        'IR_Prob': ir_probs
    })

    # Create a DataFrame for EI given IR probabilities
    ei_df = pd.DataFrame(ei_probs, columns=['EI_given_IR_low', 'EI_given_IR_medium', 'EI_given_IR_high'])
    ei_df['EI_State'] = ['poor', 'average', 'good']

    # Create a DataFrame for SP given IR and EI probabilities
    sp_df = pd.DataFrame(sp_probs, columns=[
        'SP_given_IR_low_EI_poor', 'SP_given_IR_low_EI_average', 'SP_given_IR_low_EI_good',
        'SP_given_IR_medium_EI_poor', 'SP_given_IR_medium_EI_average', 'SP_given_IR_medium_EI_good',
        'SP_given_IR_high_EI_poor', 'SP_given_IR_high_EI_average', 'SP_given_IR_high_EI_good'
    ])
    sp_df['SP_State'] = ['decrease', 'stable', 'increase']

    # Combine all data into a single DataFrame
    combined_df = pd.concat([ir_df, ei_df, sp_df], axis=1)

    # Save the combined DataFrame as a single CSV file
    combined_df.to_csv(filename, index=False)

# Save outcomes in a CSV file
def save_outcomes(data_dense, filename):
    data_dense['IR'] = data_dense['IR'].map(ir_map)
    data_dense['EI'] = data_dense['EI'].map(ei_map)
    data_dense['SP'] = data_dense['SP'].map(sp_map)
    data_dense.to_csv(filename, index=False)

# Generate datasets for different sample sizes for the dense model
sample_sizes = range(1000, 10500, 1000)
for size in sample_sizes:
    # Generate the CPDs
    ir_probs, ei_given_ir_probs, sp_probs_reshaped = generate_cpds()

    # Define CPDs for the dense model
    cpd_ir = TabularCPD(variable='IR', variable_card=3, values=[[ir_probs[0]], [ir_probs[1]], [ir_probs[2]]])
    cpd_ei_dense = TabularCPD(variable='EI', variable_card=3,
                              values=ei_given_ir_probs,
                              evidence=['IR'], evidence_card=[3])
    cpd_sp_dense = TabularCPD(variable='SP', variable_card=3,
                              values=sp_probs_reshaped,
                              evidence=['IR', 'EI'], evidence_card=[3, 3])

    dense_model.add_cpds(cpd_ir, cpd_ei_dense, cpd_sp_dense)

    # Check if the model is valid
    assert dense_model.check_model()

    # Generate samples
    sampler_dense = BayesianModelSampling(dense_model)
    data_dense = sampler_dense.forward_sample(size=size)

    # Save probabilities in one file
    save_probabilities(ir_probs, ei_given_ir_probs, sp_probs_reshaped, f'probabilities_dense_{size}.csv')

    # Save outcomes (low, medium, high) in another file
    save_outcomes(data_dense, f'outcomes_dense_{size}.csv')

# Notify the user that the process is done
print("Data generation and saving complete for the dense model!")

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

...

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

Data generation and saving complete for the dense model!


# Hypothesis Model: 500, 1000, 1500, ..., 10000 Samples (dense) 1 hidden Layer, 20 Neurons Relu

In [4]:
# Sample sizes to loop through
sample_sizes = range(1000, 10500, 1000)

# Define the Neural Network architecture
def create_nn_model(hidden_layers=1, nodes_per_layer=20):
    model = models.Sequential()

    # Input layer (2 input features: IR_encoded and EI_encoded)
    model.add(layers.InputLayer(input_shape=(2,)))

    # Hidden layers
    for layer_num in range(hidden_layers):
        model.add(layers.Dense(nodes_per_layer, activation='relu', name=f"hidden_layer_{layer_num + 1}"))

    # Output layer (3 classes: decrease, stable, increase)
    model.add(layers.Dense(3, activation='softmax', name="output_layer"))

    # Compile the model
    model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

    return model

# Loop through each sample size
for size in sample_sizes:
    # Load data for the current sample size (adjust the file paths if necessary)
    outcomes_file = f'outcomes_dense_{size}.csv'

    df = pd.read_csv(outcomes_file)

    # Manually encode categorical variables for IR, EI, and SP
    ir_map = {'low': 0, 'medium': 1, 'high': 2}
    ei_map = {'poor': 0, 'average': 1, 'good': 2}
    sp_map = {'decrease': 0, 'stable': 1, 'increase': 2}

    df['IR_encoded'] = df['IR'].map(ir_map)
    df['EI_encoded'] = df['EI'].map(ei_map)
    df['SP_encoded'] = df['SP'].map(sp_map)

    # Features (IR and EI) and labels (SP)
    X = df[['IR_encoded', 'EI_encoded']]
    y = df['SP_encoded']

    # Refresh the data split for each iteration
    X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, shuffle=False, random_state=42)
    X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, shuffle=False, random_state=42)

    # Show split confirmation
    print(f"\nSample size: {size}")
    print("Training Data:", X_train.shape, y_train.shape)
    print("Validation Data:", X_val.shape, y_val.shape)
    print("Test Data:", X_test.shape, y_test.shape)

    # Create the Neural Network model
    nn_model = create_nn_model(hidden_layers=1, nodes_per_layer=10)

    # Train the model
    history = nn_model.fit(X_train, y_train,
                           epochs=50,
                           batch_size=32,
                           validation_data=(X_val, y_val),
                           verbose=0)  # Set verbose=0 to avoid too much output

    # Evaluate on the validation set
    val_loss, val_accuracy = nn_model.evaluate(X_val, y_val, verbose=0)
    print(f"Validation Accuracy for {size} samples: {val_accuracy:.4f}")

    # Evaluate on the test set
    test_loss, test_accuracy = nn_model.evaluate(X_test, y_test, verbose=0)
    print(f"Test Accuracy for {size} samples: {test_accuracy:.4f}")

    # Make predictions on the test set
    predictions = nn_model.predict(X_test)

    # Convert the predicted probabilities to class labels
    predicted_classes = predictions.argmax(axis=1)

    # Create a list to map integers back to the original SP labels
    sp_reverse_map = ['decrease', 'stable', 'increase']

    # Convert the predicted classes to the original labels
    predicted_labels = [sp_reverse_map[label] for label in predicted_classes]

    # Create a DataFrame for the predicted probabilities
    probs_df = pd.DataFrame(predictions, columns=['Prob_decrease', 'Prob_stable', 'Prob_increase'])

    # Output the IR, EI, predicted SP, and the NN probabilities
    result_df = pd.DataFrame({
        'IR': df['IR'][:len(predicted_labels)],  # IR column from the original dataframe
        'EI': df['EI'][:len(predicted_labels)],  # EI column from the original dataframe
        'Predicted_SP': predicted_labels         # Predicted SP labels
    })

    # Combine the result with the predicted probabilities
    combined_df = pd.concat([result_df, probs_df.reset_index(drop=True)], axis=1)

    # Show the first few rows of the results for this sample size
    print(f"\nPredicted Results and Probabilities for {size} samples (First 5 rows):")
    print(combined_df.head(15))

# After the loop is done, print this message
print("\nLooping through all sample sizes complete!")

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/input_layer.py:26: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(



Sample size: 1000
Training Data: (700, 2) (700,)
Validation Data: (150, 2) (150,)
Test Data: (150, 2) (150,)
Validation Accuracy for 1000 samples: 0.5067
Test Accuracy for 1000 samples: 0.5000
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step

Predicted Results and Probabilities for 1000 samples (First 5 rows):
      IR       EI Predicted_SP  Prob_decrease  Prob_stable  Prob_increase
0    low     good     decrease       0.442138     0.281528       0.276334
1    low     good       stable       0.346824     0.489251       0.163925
2   high  average     decrease       0.444290     0.370255       0.185455
3    low     poor       stable       0.346824     0.489251       0.163925
4   high     good       stable       0.346824     0.489251       0.163925
5   high  average     decrease       0.378509     0.351758       0.269734
6   high     good     decrease       0.442138     0.281528       0.276334
7    low     good     decrease       0.450406     0.418139       0.131455
8   high     poor     decrease  

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/input_layer.py:26: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


Validation Accuracy for 2000 samples: 0.4667
Test Accuracy for 2000 samples: 0.4300
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 

Predicted Results and Probabilities for 2000 samples (First 5 rows):
        IR       EI Predicted_SP  Prob_decrease  Prob_stable  Prob_increase
0   medium     poor     decrease       0.385673     0.337932       0.276395
1   medium     good       stable       0.343182     0.464438       0.192379
2      low     good       stable       0.369172     0.412777       0.218051
3   medium     poor     increase       0.402617     0.054553       0.542830
4   medium  average     decrease       0.417982     0.225731       0.356288
5      low     good       stable       0.343182     0.464438       0.192379
6   medium     poor     decrease       0.385673     0.337932       0.276395
7      low     good     decrease       0.417982     0.225731       0.356288
8   medium     good       stable       0.343182     0.464438       0.192379
9   medium     poor     decrease       0.44585

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/input_layer.py:26: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


...

Validation Accuracy for 8000 samples: 0.4967
Test Accuracy for 8000 samples: 0.4875
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step

Predicted Results and Probabilities for 8000 samples (First 5 rows):
        IR       EI Predicted_SP  Prob_decrease  Prob_stable  Prob_increase
0   medium  average     decrease       0.467418     0.068935       0.463647
1     high     good       stable       0.366324     0.427867       0.205809
2   medium  average       stable       0.310285     0.542847       0.146868
3      low     poor     increase       0.194661     0.378762       0.426577
4     high  average       stable       0.310285     0.542847       0.146868
5   medium     good     decrease       0.467418     0.068935       0.463647
6   medium     good     increase       0.194661     0.378762       0.426577
7     high     poor       stable       0.366324     0.427867       0.205809
8   medium     good     increase       0.254404     0.296624       0.448972
9   medium     good       stable       0.366324

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/input_layer.py:26: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


Validation Accuracy for 9000 samples: 0.4541
Test Accuracy for 9000 samples: 0.4719
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step

Predicted Results and Probabilities for 9000 samples (First 5 rows):
        IR       EI Predicted_SP  Prob_decrease  Prob_stable  Prob_increase
0   medium     good     increase       0.262058     0.344555       0.393387
1     high  average       stable       0.390330     0.534253       0.075416
2      low     poor     increase       0.262058     0.344555       0.393387
3   medium     poor       stable       0.289628     0.549872       0.160500
4   medium  average     increase       0.218974     0.301836       0.479190
5   medium     poor       stable       0.390330     0.534253       0.075416
6   medium     poor     increase       0.156446     0.121375       0.722179
7      low     poor       stable       0.289628     0.549872       0.160500
8   medium     good       stable       0.289628     0.549872       0.160500
9   medium     poor     increase       0.218974

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/input_layer.py:26: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


Validation Accuracy for 10000 samples: 0.5847
Test Accuracy for 10000 samples: 0.5807
47/47 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step

Predicted Results and Probabilities for 10000 samples (First 5 rows):
        IR       EI Predicted_SP  Prob_decrease  Prob_stable  Prob_increase
0   medium     poor     increase       0.298701     0.305421       0.395878
1   medium     poor       stable       0.149282     0.771987       0.078731
2     high  average       stable       0.227560     0.600968       0.171472
3   medium     poor       stable       0.149282     0.771987       0.078731
4   medium     good       stable       0.272346     0.489474       0.238180
5   medium     good     increase       0.298701     0.305421       0.395878
6   medium     poor       stable       0.272346     0.489474       0.238180
7     high  average       stable       0.149282     0.771987       0.078731
8   medium     good     increase       0.298701     0.305421       0.395878
9      low  average     decrease       0.559

# K-L Divergence NN Dense Data

In [5]:
# Define the function to save K-L divergence and std dev to a file
def save_kl_divergence(sample_size, kl_div_value, std_kl_div_value, first_run=False):
    file_name = 'kl_div_NN_1_20_dense.csv'

    # If it's the first run, write the headers
    if first_run:
        with open(file_name, 'w') as f:
            f.write('Size,NN_Dense_1_20_Entropy,Std_Dev\n')  # Write headers for the CSV

    # Append the K-L divergence and std dev for this sample size to the file
    with open(file_name, 'a') as f:
        f.write(f"{sample_size},{kl_div_value:.4f},{std_kl_div_value:.4f}\n")

# Sample sizes to loop through
sample_sizes = range(1000, 10500, 1000)  # Loop from 500 to 10,000 in increments of 500

# Loop through each sample size
first_run = True  # Flag to indicate the first run for writing headers

for sample_size in sample_sizes:
    # Load the ground truth probabilities for the current sample size
    ground_truth_probs_file = f'probabilities_dense_{sample_size}.csv'
    df_gt_probs = pd.read_csv(ground_truth_probs_file)

    # Placeholder to store K-L divergence values
    kl_divergences = []

    # Load the predictions from memory (already generated by the NN)
    outcomes_file = f'outcomes_dense_{sample_size}.csv'
    df = pd.read_csv(outcomes_file)

    # Manually encode categorical variables for IR and EI
    ir_map = {'low': 0, 'medium': 1, 'high': 2}
    ei_map = {'poor': 0, 'average': 1, 'good': 2}

    df['IR_encoded'] = df['IR'].map(ir_map)
    df['EI_encoded'] = df['EI'].map(ei_map)

    # Assuming predictions were saved in memory from the NN run
    predictions = nn_model.predict(df[['IR_encoded', 'EI_encoded']])

    # Loop through the test set predictions
    for i in range(len(df)):
        # Neural Network predicted probabilities for SP (decrease, stable, increase)
        predicted_probs = predictions[i]

        # Get the IR and EI values for the current sample
        ir_value = df.iloc[i]['IR_encoded']
        ei_value = df.iloc[i]['EI_encoded']

        # Map encoded values back to original labels
        ir_value = {0: 'low', 1: 'medium', 2: 'high'}[ir_value]
        ei_value = {0: 'poor', 1: 'average', 2: 'good'}[ei_value]

        # Get the corresponding ground truth probabilities for SP given IR and EI
        col_prefix = f'SP_given_IR_{ir_value}_EI_{ei_value}'
        ground_truth_probs = df_gt_probs.filter(like=col_prefix).values.flatten()

        # Ensure the probabilities are non-zero to avoid division by zero
        epsilon = 1e-10
        ground_truth_probs = np.clip(ground_truth_probs, epsilon, 1)

        # Compute K-L divergence (Neural Network vs Ground Truth)
        kl_div = entropy(predicted_probs, ground_truth_probs)
        kl_divergences.append(kl_div)

    # Calculate the average K-L divergence over all samples for the current sample size
    average_kl_divergence = np.mean(kl_divergences)

    # Calculate the standard deviation of the K-L divergence values
    std_kl_divergence = np.std(kl_divergences)

    # Save the K-L divergence and std dev value to a CSV file
    save_kl_divergence(sample_size, average_kl_divergence, std_kl_divergence, first_run=first_run)

    # Print confirmation
    print(f"Average K-L Divergence for {sample_size} samples: {average_kl_divergence:.4f}, Std Dev: {std_kl_divergence:.4f}")

    # After the first run, set `first_run` to False
    first_run = False

# Once all sample sizes are processed, the K-L divergences and std devs will be saved in 'kl_div_NN_1_20_dense.csv'
print("\nK-L divergence and standard deviation calculations complete.")

32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
Average K-L Divergence for 1000 samples: 0.4498, Std Dev: 0.6371
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
Average K-L Divergence for 2000 samples: 0.4463, Std Dev: 0.4856
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
Average K-L Divergence for 3000 samples: 0.3096, Std Dev: 0.2652
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
Average K-L Divergence for 4000 samples: 0.3156, Std Dev: 0.2472
157/157 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
Average K-L Divergence for 5000 samples: 0.3780, Std Dev: 0.3365
188/188 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
Average K-L Divergence for 6000 samples: 0.2910, Std Dev: 0.2399
219/219 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
Average K-L Divergence for 7000 samples: 0.7457, Std Dev: 0.7001
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
Average K-L Divergence for 8000 samples: 0.2362, Std Dev: 0.3580
282/282 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
Average K-L Divergence for 9000 samples: 0.3130, Std Dev: 0.2123
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
Average K-